# Garbage Classification Prototype

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms



In [3]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch.utils.data

# Set the path to your data folder
data_path = "C:/Users/legen/Eco_Bins/garbage_classification/all_classes"

# Define the transforms to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224 x 224
    transforms.ToTensor(),  # Convert the images to PyTorch tensors
])

# Use ImageFolder to load the data
dataset = ImageFolder(root=data_path, transform=transform)

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


In [8]:
import os
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader

data_dir = "C:/Users/legen/Eco_Bins/garbage_classification/all_classes"
dataset = ImageFolder(data_dir)

# Split dataset into train, val and test sets
val_size = 0.2
test_size = 0.1
train_size = 1 - val_size - test_size

train_len = int(train_size * len(dataset))
val_len = int(val_size * len(dataset))
test_len = len(dataset) - train_len - val_len

train_set, val_set, test_set = random_split(dataset, [train_len, val_len, test_len])

# Set up data loaders
batch_size = 32

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=4)



In [9]:
class ResNetImproved(nn.Module):
    def __init__(self, num_classes=6, dropout=0.3):
        super(ResNetImproved, self).__init__()
        resnet = models.resnet50(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(512, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [20]:
# Define the training loop function
def train(epochs, model, optimizer, criterion, train_loader, validation_loader):
    history = []
    for epoch in range(epochs):
        train_loss = 0.0
        model.train() # Set the model to training mode
        for i, (inputs, targets) in enumerate(train_loader):
            # Move inputs and targets to device
            inputs, targets = inputs.to(device), targets.to(device)
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            # Backward pass
            loss.backward()
            # Update weights
            optimizer.step()
            # Update training loss
            train_loss += loss.item() * inputs.size(0)
        
        # Calculate validation loss and accuracy
        val_loss, val_acc = evaluate(model, criterion, validation_loader)
        
        # Update the history dictionary
        history.append({'epoch': epoch+1, 'train_loss': train_loss/len(train_loader.dataset), 'val_loss': val_loss, 'val_acc': val_acc})
        
        # Print progress
        print(f"Epoch: {epoch+1}/{epochs} Train Loss: {train_loss/len(train_loader.dataset)} Val Loss: {val_loss} Val Acc: {val_acc}")
    
    return history


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1000),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Define the evaluation function
def evaluate(model, criterion, data_loader):
    loss = 0.0
    accuracy = 0.0
    model.eval() # Set the model to evaluation mode
    with torch.no_grad():
        for inputs, targets in data_loader:
            # Move inputs and targets to device
            inputs, targets = inputs.to(device), targets.to(device)
            # Forward pass
            outputs = model(inputs)
            loss += criterion(outputs, targets).item() * inputs.size(0)
            # Calculate accuracy
            _, preds = torch.max(outputs, dim=1)
            accuracy += torch.sum(preds == targets).item()
    loss /= len(data_loader.dataset)
    accuracy /= len(data_loader.dataset)
    return loss, accuracy

# Define the training loop function
def train(epochs, model, optimizer, criterion, train_loader, validation_loader):
    best_val_acc = 0.0
    for epoch in range(epochs):
        train_loss = 0.0
        model.train() # Set the model to training mode
        for inputs, targets in train_loader:
            # Move inputs and targets to device
            inputs, targets = inputs.to(device), targets.to(device)
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            # Backward pass
            loss.backward()
            # Update weights
            optimizer.step()
            # Update training loss
            train_loss += loss.item() * inputs.size(0)
        
        # Calculate validation loss and accuracy
        val_loss, val_acc = evaluate


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_data_loaders(batch_size):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    try:
        train_dataset = datasets.ImageFolder(root='C:/Users/legen/Eco_Bins/garbage_classification/all_classes', transform=transform)
    except FileNotFoundError:
        print("Could not find 'train' directory. Make sure the directory is located in the same directory as this script.")
        return None, None
    
    validation_dataset = datasets.ImageFolder(root='C:/Users/legen/Eco_Bins/garbage_classification/all_classes', transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, validation_loader

def train(epochs, model, optimizer, criterion, train_loader, validation_loader):
    history = []
    for epoch in range(epochs):
        train_loss = 0.0
        model.train()
        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        val_loss, val_acc = evaluate(model, criterion, validation_loader)
        
        history.append({'epoch': epoch+1, 'train_loss': train_loss/len(train_loader.dataset), 'val_loss': val_loss, 'val_acc': val_acc})
        
        print(f"Epoch: {epoch+1}/{epochs} Train Loss: {train_loss/len(train_loader.dataset)} Val Loss: {val_loss} Val Acc: {val_acc}")
    
    return history

def evaluate(model, criterion, data_loader):
    model.eval()
    loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss += criterion(outputs, targets).item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
    
    return loss/len(data_loader.dataset), correct/len(data_loader.dataset)

# Define the model
model = models.resnet18(weights = 'ResNet18_Weights.DEFAULT')
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

# Define the optimizer and criterion
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [30]:
# Define the hyperparameters
epochs = 10
learning_rate = 0.001
batch_size = 32

# Create the data loaders
train_loader, validation_loader = get_data_loaders(batch_size=batch_size)

# Create the model
model = models.resnet18(weights = 'ResNet18_Weights.DEFAULT')
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) # Replace the last fully-connected layer
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim


In [32]:
epochs = 10
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_loader, validation_loader = get_data_loaders(batch_size=32)

history = train(epochs, model, optimizer, criterion, train_loader, validation_loader)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
